In [21]:
import requests
requests.packages.urllib3.disable_warnings()
import json
import numpy as np

In [22]:
def makeRequest(data: dict, modelName:str = "model", port: int = 8443) -> requests.Response:
    url = f"https://127.0.0.1:{port}/predictions/{modelName}"
    r = requests.get(url, data=json.dumps({"data": data}), verify=False)
    return r

In [24]:
testData = np.random.rand(2,2).tolist()
r = makeRequest(testData, "model")
print(r.text)

[
  [
    0.5342158079147339,
    0.8330368995666504
  ],
  [
    0.5346289873123169,
    0.8347043395042419
  ]
]
